In [32]:
import pandas as pd
import numpy as np

''' 
# Play with the seed if need be
np.random.seed(42)
num_stocks = 80
industries = ['Tech', 'Healthcare', 'Finance', 'Energy', 'Consumer']

data = {
    'Stock Name': [f'Stock_{i+1}' for i in range(num_stocks)],
    'TSR': np.random.uniform(5, 15, num_stocks),  # TSR between 5% and 15%
    'Grade': np.random.uniform(50, 100, num_stocks),  # Grade between 50 and 100
    'Beta Value': np.random.uniform(0.5, 1.5, num_stocks),  # Beta between 0.5 and 1.5
    'Industry': np.random.choice(industries, num_stocks)  # Randomly assign industry
}

dummy_data = pd.DataFrame(data)
'''

" \n# Play with the seed if need be\nnp.random.seed(42)\nnum_stocks = 80\nindustries = ['Tech', 'Healthcare', 'Finance', 'Energy', 'Consumer']\n\ndata = {\n    'Stock Name': [f'Stock_{i+1}' for i in range(num_stocks)],\n    'TSR': np.random.uniform(5, 15, num_stocks),  # TSR between 5% and 15%\n    'Grade': np.random.uniform(50, 100, num_stocks),  # Grade between 50 and 100\n    'Beta Value': np.random.uniform(0.5, 1.5, num_stocks),  # Beta between 0.5 and 1.5\n    'Industry': np.random.choice(industries, num_stocks)  # Randomly assign industry\n}\n\ndummy_data = pd.DataFrame(data)\n"

In [33]:
# load your data 
# something = pd.read_csv('something.csv')

companies_selected = pd.read_excel('Companies.xlsx')

In [34]:
#since TSR is in % we will grade wight in % as well
grade_weight = {"A": 90, "B": 80, "C": 70, "D": 60, "E": 50, "F": 40}
companies_selected['GradeWeight'] = companies_selected['Grade'].map(grade_weight)

In [35]:
def calculate_grade_score(data, x):
    return (1 - x) * data['TSR'] + x * data['GradeWeight']

In [36]:
# default values set as: 16% for industry allocation and 4.5% mac allocation
# beta value for potfolio should be below 1 
def monte_carlo_simulation(data, max_industry_allocation=0.16, max_stock_allocation=0.045, target_beta=1.0, iterations=100):
    
    best_portfolio = None
    best_portfolio_beta = float('inf')
    
    for _ in range(iterations):
        # The weightage we are giving x will range from 0.55 to 0.7
        # This will give us cases where our grade is not the main factor and TSR is which provides us with different scores on multiple iterations
        x = np.random.uniform(0.55, 0.7)
        
        # execute the function mention above
        data['Grade Score'] = data.apply(calculate_grade_score, axis=1, args=(x,))
        
        # obtain a weight scross all the investments. Basically getting a ratio so values for the weights will be between 0 and 1 
        # basic normalisation 
        data['Weight'] = data['Grade Score'] / data['Grade Score'].sum()
        
        # Lets focus on each industry
        # what is the current weight allocated to each industry
        data['Industry Allocation'] = data.groupby('Industry')['Weight'].transform('sum')
        data['Allocation_by_restriction'] = np.where(data['Industry Allocation'] > max_industry_allocation, max_industry_allocation, data['Industry Allocation'])
      
        data['Normalised Weight'] = np.where(data['Allocation_by_restriction'] == 0.16, (data['Weight']/data['Industry Allocation'])*0.16, data['Weight'])
        
        # NEED TO ADD WEIGHT FOR THE MAX ALLOCATION

        #Calculate Portfolio Beta
        portfolio_beta = (data['Beta Value'] * data['Normalised Weight']).sum()
        
        if portfolio_beta < target_beta:
            best_portfolio = data[['Company Name', 'Industry', 'Weight', 'Normalised Weight', 'Beta Value']]
            best_portfolio_beta = portfolio_beta
            break  

    return portfolio_beta, data

In [37]:
final_beta,ideal_portfolio = monte_carlo_simulation(companies_selected)

In [38]:
final_beta

0.2655497089816722

In [39]:
ideal_portfolio['Normalised Weight'].sum()

0.31999999999999995

In [40]:
ideal_portfolio.groupby('Industry')['Normalised Weight'].sum()

Industry
Finance      0.16
Utilities    0.16
Name: Normalised Weight, dtype: float64

In [41]:
ideal_portfolio['final_weight']  = (ideal_portfolio['Normalised Weight']/ideal_portfolio['Normalised Weight'].sum())*100

In [45]:
round(ideal_portfolio['final_weight'].sum(),2) == 100.00

True

In [47]:
ideal_portfolio

,Company Name,Industry,Beta Value,Grade,TSR,GradeWeight,Grade Score,Weight,Industry Allocation,Allocation_by_restriction,Normalised Weight,final_weight
0,NextEra Energy (NEE),Utilities,0.49,B,53.47,80,71.376467,0.042347,0.393523,0.16,0.017218,5.380518
1,Duke Energy (DUK),Utilities,0.38,C,57.51,70,65.940146,0.039122,0.393523,0.16,0.015906,4.970716
2,Atmos Energy (ATO),Utilities,0.50,B,54.13,80,71.590998,0.042474,0.393523,0.16,0.017269,5.396690
3,Avangrid (AGR),Utilities,0.34,A,-8.57,90,57.959981,0.034387,0.393523,0.16,0.013981,4.369154
4,Otter Tail (OTTR),Utilities,0.69,A,87.72,90,89.258890,0.052957,0.393523,0.16,0.021531,6.728535
5,Edison International (EIX),Utilities,0.80,B,56.80,80,72.458878,0.042989,0.393523,0.16,0.017479,5.462113
6,Portland General Electric (POR),Utilities,0.55,A,5.97,90,62.686185,0.037191,0.393523,0.16,0.015121,4.725425
7,IDACORP (IDA),Utilities,0.60,B,33.27,80,64.810489,0.038452,0.393523,0.16,0.015634,4.885560
8,AES Corporation (AES),Utilities,1.10,B,-9.25,80,50.989432,0.030252,0.393523,0.16,0.012300,3.843698
9,American Electric Power (AEP),Utilities,0.60,C,27.59,70,56.214698,0.033352,0.393523,0.16,0.013560,4.237590
